In [3]:
import random
import math
from random import choice
import json
import time

In [59]:
with open("input-data.json", "r") as f:
    config_data = json.load(f)

In [61]:
class Patch:
    def __init__(self, index, climate_zone, soil_quality, config):
        self.index = index
        self.climate_zone = climate_zone
        self.soil_quality = soil_quality
        self.soil_factor = config["SOILS"].get(soil_quality, 0)
        self.grass = config.get("GRASS_START", 0)
        self.max_grass = config.get("GRASS_MAX", 1)
        self.temperature = 0  # Начальная температура

    def grow_grass(self, grass_speed):
        # Используем сам атрибут temperature объекта Patch, не передаем ее как аргумент
        delta = self.soil_factor * self.temperature * grass_speed
        self.grass = min(self.grass + delta, self.max_grass)


In [63]:
class World:
    def __init__(self, config_data):
        self.config_data = config_data
        self.length = config_data["LENGTH"]
        self.season_length = config_data["SEASON_LENGTH"]
        self.grow_speed = config_data["GROW_SPEED"]

        self.patches = []
        self.cows = []  # Живые коровы
        self.new_cows = []  # Новые потомки

        self.temperature_patterns = config_data["CLIMATES"]

        self.current_tick = 0
        self.init_patches()

    def init_patches(self):
        # Для каждого участка формируем климат
        for i in range(self.length):
            if 1 <= i <= 1000 or 5001 <= i <= 6000:
                climate = 'Polar'
            elif 1001 <= i <= 2000 or 4001 <= i <= 5000 or 6001 <= i <= 7000 or 9001 <= i <= 10000:
                climate = 'Middle'
            elif 2001 <= i <= 4000 or 7001 <= i <= 9000:
                climate = 'Equator'
            else:
                climate = 'Polar'

            # Формируем почву
            if 501 <= i < 3500:  # Для оазиса
                soil = self.get_soil_for_oasis(i)
            else:
                soil = 'desert'

            patch = Patch(i, climate, soil, self.config_data)
            self.patches.append(patch)

    def get_soil_for_oasis(self, cell_number):
        # Оазис с разными типами почвы
        soil_types = ['steppe', 'field', 'forest']
        return soil_types[(cell_number - 501) // 500 % len(soil_types)]  # Почва меняется каждые 500 клеток

    def update_season(self):
        season_tick = self.current_tick % 100  
        season = (season_tick // self.season_length) % 4
        
        for patch in self.patches:
            pattern = self.temperature_patterns.get(patch.climate_zone, [1, 1, 1, 1])
            patch.temperature = pattern[season]

    def update(self):
        self.update_season()
        for patch in self.patches:
            patch.grow_grass(self.grow_speed)
        self.current_tick += 1


In [64]:
class Cow:
    def __init__(self, position, birth_tick, config, parents_qty=1):
        self.position = position
        self.energy = config["ATP_START"]
        self.raw_eat = config["RAW_EAT_START"]

        self.size = random.randint(5, 10)
        self.speed = random.randint(5, 10)
        self.metabolism = random.randint(1, 3)
        self.sight = random.randint(5, 10)
        self.herding = random.randint(1, 10)

        self.birth_tick = birth_tick
        self.death_tick = None
        self.T0 = config["MAX_AGE"]
        self.parents_qty = parents_qty
        
        self.age_min = config["AGE_MIN"]
        self.age_mult = config["AGE_MULT"]
        self.age_t_weight = config["AGE_T_WEIGHT"]
        self.chance_num = config["CHANCE_NUMERATOR"]
        self.chance_den = config["CHANCE_DENOMINATOR"]
        self.breed_type = config["BREED_TYPE"]
        self.birth_count = config["BIRTH_COUNT"]

        self.config = config  # сохраняем для потомства

    def move(self, world):
        min_pos = max(0, self.position - self.sight)
        max_pos = min(world.length, self.position + self.sight + 1)

        best_pos = self.position
        best_grass = world.patches[self.position].grass

        for pos in range(min_pos, max_pos):
            patch = world.patches[pos]
            cow_count = sum(1 for cow in world.cows if cow.position == pos and cow != self)
            if cow_count > self.herding:
                continue
            if patch.grass > best_grass:
                best_grass = patch.grass
                best_pos = pos

        if best_pos != self.position:
            distance = abs(best_pos - self.position)
            if distance <= self.speed:
                self.position = best_pos

    def eat(self, patch):
        eat_amount = min(patch.grass, self.size * 2)
        self.energy += eat_amount * self.metabolism
        patch.grass -= eat_amount


    def update(self, current_tick, world):
        age = current_tick - self.birth_tick
        self.energy -= self.metabolism * (self.size + self.speed) 
    
        patch = world.patches[self.position]
        self.eat(patch)
        self.move(world)
    
        # Проверка возраста и энергии для размножения
        if age >= 0.2 * self.T0 and self.energy >= self.config["BIRTH_ENERGY"]:
            # Размножение
            children = []
            max_children = self.birth_count
    
            if self.breed_type == 1:  # бесполое размножение
                for _ in range(min(self.birth_count, max_children)):  # Ограничиваем количество потомков
                    children.append(self.clone(current_tick))
            elif self.breed_type == 2:  # половое размножение
                for cow in world.cows:
                    if cow != self and cow.position == self.position:
                        for _ in range(min(self.birth_count, max_children)):  # Ограничиваем количество потомков
                            children.append(self.reproduce(cow, current_tick))
                        break
    
            world.new_cows.extend(children)
            self.energy -= self.config["BIRTH_ENERGY"]  
    
        return self.die(age)

    
    def die(self, age):
        if self.energy <= 0 or age >= self.T0:
            self.death_tick = self.birth_tick + age
            return True
        
        if age >= self.age_min:
            T = self.age_mult * (age - self.age_min)
            p_death = self.age_t_weight * math.exp(T)
    
            if random.random() < p_death:
                self.death_tick = self.birth_tick + age
                return True
        
        return False




    def clone(self, current_tick):
        child = Cow(
            position=self.position,
            birth_tick=current_tick,
            config=self.config,
            parents_qty=1
        )

        # мутации
        child.size = max(1, self.size + random.choice([-1, 0, 1]))
        child.speed = max(1, self.speed + random.choice([-1, 0, 1]))
        child.metabolism = max(1, self.metabolism + random.choice([-1, 0, 1]))
        child.sight = max(1, self.sight + random.choice([-1, 0, 1]))
        child.herding = max(1, self.herding + random.choice([-1, 0, 1]))
        return child

    def reproduce(self, partner, current_tick):
        num_offspring = random.randint(1, self.config["BIRTH_COUNT"])
        offspring_list = []
    
        for _ in range(num_offspring):
            child = Cow(
                position=self.position,
                birth_tick=current_tick,
                config=self.config,
                parents_qty=2
            )
    
            # среднее значение параметров от родителей + мутации
            child.size = max(1, (self.size + partner.size) // 2 + random.choice([-1, 0, 1]))
            child.speed = max(1, (self.speed + partner.speed) // 2 + random.choice([-1, 0, 1]))
            child.metabolism = max(1, (self.metabolism + partner.metabolism) // 2 + random.choice([-1, 0, 1]))
            child.sight = max(1, (self.sight + partner.sight) // 2 + random.choice([-1, 0, 1]))
            child.herding = max(1, (self.herding + partner.herding) // 2 + random.choice([-1, 0, 1]))

    
            offspring_list.append(child)
    
        return offspring_list

In [65]:
class Simulation:
    def __init__(self, config_data):
        self.config_data = config_data
        self.world = World(config_data)
        self.ticks = config_data["TIME"]
        self.current_tick = 0

        # Начальная популяция коров
        self.cows = self.create_initial_cows()

        # Логирование
        self.log_file = open("simulation_log.txt", "w")
        self.log_initial_stats()
        self.log_file.write("Tick, Total cows, Average Age, Total Grass\n")

    def create_initial_cows(self):
        cows = []
        num_groups = self.config_data["COWS_COUNT"] // 10
        
        for _ in range(num_groups):
            position = random.randint(501, 3500)
            
            for _ in range(10):
                birth_tick = 0
                herding = random.randint(1, 10)
                
                cow = Cow(
                    position=position,
                    birth_tick=birth_tick,
                    config=self.config_data,
                    parents_qty=1
                )
                cow.herding = herding
                cows.append(cow)
        
        return cows

    def log(self, message=""):
        if message:
            self.log_file.write(f"{message}\n")
        
        num_cows = len(self.cows)
        total_age = sum([max(0, self.current_tick - cow.birth_tick) for cow in self.cows])
        avg_age = total_age // num_cows if num_cows > 0 else 0
        total_grass = sum([int(patch.grass) for patch in self.world.patches])
        self.log_file.write(f"{self.current_tick}, {num_cows}, {avg_age}, {total_grass}\n")

    def log_initial_stats(self):
        self.log_file.write("Initial Stats:\n")
        soil_count = {'desert': 0, 'steppe': 0, 'field': 0, 'forest': 0}
        initial_grass = 0.0

        for patch in self.world.patches:
            soil_count[patch.soil_quality] += 1
            initial_grass += patch.grass

        self.log_file.write(f"Initial distribution of soil types: {soil_count}\n")
        self.log_file.write(f"Total initial grass: {initial_grass:.2f}\n")

    def run(self):
        start_time = time.time()

        for tick in range(self.ticks):
            self.current_tick = tick
            self.world.update()

            new_cows = self.cows[:]
            self.cows.clear()

            for cow in new_cows:
                if not cow.update(self.current_tick, self.world):
                    self.cows.append(cow)

            self.cows.extend(self.world.new_cows)
            self.world.new_cows.clear()

            if len(self.cows) == 0:
                self.log_file.write("All cows are dead. Simulation stopped.\n")
                break

            self.log()

        else:
            self.log_file.write("Time limit reached. Simulation stopped.\n")

        end_time = time.time()
        duration = end_time - start_time
        self.log_file.write(f"\nSimulation completed in {duration:.2f} seconds.\n")
        self.log_file.close()


In [60]:
simulation = Simulation(config_data)

simulation.run()

print("Simulation completed. Logs saved to 'simulation_log.txt'.")


Simulation completed. Logs saved to 'simulation_log.txt'.
